# Analysis of Airbnb prices in European cities

## Load and transform data

In [1]:
#!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client pandas

In [2]:
import numpy as np
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Set display option to show all columns
pd.set_option('display.max_columns', None)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Load all the data using API

In [3]:
# Google Sheets document URL and spreadsheet ID
spreadsheet_id = '1ecopK6oyyb4d_7-QLrCr8YlgFrCetHU7-VQfnYej7JY'

# Path to downloaded Service Account JSON file
json_key_file = 'banded-anvil-309310-3642b97f7801.json'

# Authenticate using OAuth 2.0 JSON file
creds = service_account.Credentials.from_service_account_file(
    json_key_file,
    scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"]
)
service = build('sheets', 'v4', credentials=creds)

# Get the list of all sheets in the spreadsheet
spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
sheet_names = [sheet['properties']['title'] for sheet in spreadsheet['sheets']]

# Load each sheet and add city name and day type as columns
all_sheets = []
for sheet_name in sheet_names:
    # Fetch data from Google Sheets
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=sheet_name).execute()
    values = result.get('values', [])
    
    if values:
        # Convert to DataFrame
        df = pd.DataFrame(values[1:], columns=values[0])  # Assuming first row as header

        # Drop the first column which contains the index
        df = df.drop(df.columns[0], axis=1)

        # Extract city name and day type from sheet name
        city_name, day_type = sheet_name.split('_')
        df['city'] = city_name.capitalize()
        df['day_type'] = day_type.capitalize()  # Capitalize to get 'Weekdays' or 'Weekend'

        # Append DataFrame to the list
        all_sheets.append(df)

# Combine all sheets into a single DataFrame
combined_df = pd.concat(all_sheets, ignore_index=True).reset_index(drop=True)

# Display the combined DataFrame
display(combined_df.head())

,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,city,day_type
0,194.0336981,Private room,FALSE,TRUE,2,FALSE,1,0,10,93,1,5.022963798,2.539380003,78.69037927,4.166707868,98.25389587,6.846472824,4.90569,52.41772,Amsterdam,Weekdays
1,344.245776,Private room,FALSE,TRUE,4,FALSE,0,0,8,85,1,0.4883892888,0.2394039228,631.1763783,33.42120862,837.2807567,58.34292774,4.90005,52.37432,Amsterdam,Weekdays
2,264.1014224,Private room,FALSE,TRUE,2,FALSE,0,1,9,87,1,5.748311915,3.651621289,75.27587691,3.9859077,95.38695493,6.646700255,4.97512,52.36103,Amsterdam,Weekdays
3,433.529398,Private room,FALSE,TRUE,4,FALSE,0,1,9,90,2,0.3848620128,0.4398760761,493.2725344,26.11910845,875.0330976,60.97356517,4.89417,52.37663,Amsterdam,Weekdays
4,485.5529257,Private room,FALSE,TRUE,2,TRUE,0,0,10,98,1,0.5447381834,0.3186926468,552.8303244,29.272733,815.30574,56.81167696,4.90051,52.37508,Amsterdam,Weekdays


In [4]:
combined_df.columns

Index(['realSum', 'room_type', 'room_shared', 'room_private',
       'person_capacity', 'host_is_superhost', 'multi', 'biz',
       'cleanliness_rating', 'guest_satisfaction_overall', 'bedrooms', 'dist',
       'metro_dist', 'attr_index', 'attr_index_norm', 'rest_index',
       'rest_index_norm', 'lng', 'lat', 'city', 'day_type'],
      dtype='object')

In [5]:
# Convert specified columns to int64
int_columns = ['person_capacity', 'multi', 'biz', 'cleanliness_rating', 'guest_satisfaction_overall', 'bedrooms']
combined_df[int_columns] = combined_df[int_columns].astype('int64')

# Convert specified columns to float
float_columns = ['realSum', 'dist', 'metro_dist', 'attr_index', 'attr_index_norm', 'rest_index',
                 'rest_index_norm', 'lng', 'lat']
combined_df[float_columns] = combined_df[float_columns].astype('float64')

# Display the updated DataFrame and check dtypes
display(combined_df.head())

,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,city,day_type
0,194.033698,Private room,FALSE,TRUE,2,FALSE,1,0,10,93,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772,Amsterdam,Weekdays
1,344.245776,Private room,FALSE,TRUE,4,FALSE,0,0,8,85,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432,Amsterdam,Weekdays
2,264.101422,Private room,FALSE,TRUE,2,FALSE,0,1,9,87,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103,Amsterdam,Weekdays
3,433.529398,Private room,FALSE,TRUE,4,FALSE,0,1,9,90,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663,Amsterdam,Weekdays
4,485.552926,Private room,FALSE,TRUE,2,TRUE,0,0,10,98,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508,Amsterdam,Weekdays


In [6]:
# Get info about columns
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51707 entries, 0 to 51706
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   realSum                     51707 non-null  float64
 1   room_type                   51707 non-null  object 
 2   room_shared                 51707 non-null  object 
 3   room_private                51707 non-null  object 
 4   person_capacity             51707 non-null  int64  
 5   host_is_superhost           51707 non-null  object 
 6   multi                       51707 non-null  int64  
 7   biz                         51707 non-null  int64  
 8   cleanliness_rating          51707 non-null  int64  
 9   guest_satisfaction_overall  51707 non-null  int64  
 10  bedrooms                    51707 non-null  int64  
 11  dist                        51707 non-null  float64
 12  metro_dist                  51707 non-null  float64
 13  attr_index                  517

### Create column with countries and change the name of realSum to Price

In [7]:
# Get unique city names
combined_df['city'].unique()

array(['Amsterdam', 'Athens', 'Berlin', 'Barcelona', 'Budapest', 'Lisbon',
       'London', 'Paris', 'Rome', 'Vienna'], dtype=object)

In [8]:
# Define the mapping of cities to countries
city_to_country = {
    'Amsterdam': 'Netherlands',
    'Athens': 'Greece',
    'Berlin': 'Germany',
    'Barcelona': 'Spain',
    'Budapest': 'Hungary',
    'Lisbon': 'Portugal',
    'London': 'United Kingdom',
    'Paris': 'France',
    'Rome': 'Italy',
    'Vienna': 'Austria'
}

# Map the city column to the country column using the dictionary
combined_df['country'] = combined_df['city'].map(city_to_country)

# Change name of the column realSum to price
combined_df = combined_df.rename(columns={'realSum': 'price'})

combined_df.head()

,price,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,city,day_type,country
0,194.033698,Private room,FALSE,TRUE,2,FALSE,1,0,10,93,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772,Amsterdam,Weekdays,Netherlands
1,344.245776,Private room,FALSE,TRUE,4,FALSE,0,0,8,85,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432,Amsterdam,Weekdays,Netherlands
2,264.101422,Private room,FALSE,TRUE,2,FALSE,0,1,9,87,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103,Amsterdam,Weekdays,Netherlands
3,433.529398,Private room,FALSE,TRUE,4,FALSE,0,1,9,90,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663,Amsterdam,Weekdays,Netherlands
4,485.552926,Private room,FALSE,TRUE,2,TRUE,0,0,10,98,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508,Amsterdam,Weekdays,Netherlands


### Instead of having two columns of room_shared and room_private, create one with the respective categories

In [9]:
# Drop duplicated column
combined_df = combined_df.drop(columns=['room_type'])

# Define conditions
conditions = [
    (combined_df['room_shared'] == 'TRUE'),
    (combined_df['room_private'] == 'TRUE') & (combined_df['room_shared'] == 'FALSE'),
    (combined_df['room_shared'] == 'FALSE') & (combined_df['room_private'] == 'FALSE')
]

# Define the corresponding choices for each condition
choices = ['Shared', 'Private', 'Entire Apt']

# Create the new column using np.select, with the default set to 'Unknown' or leave as NaN
combined_df['room_type'] = np.select(conditions, choices, default='Unknown')

# Drop the original room_shared and room_private columns if they are no longer needed
combined_df = combined_df.drop(columns=['room_shared', 'room_private'])

# Display the updated DataFrame
display(combined_df.head())

,price,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,city,day_type,country,room_type
0,194.033698,2,FALSE,1,0,10,93,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772,Amsterdam,Weekdays,Netherlands,Private
1,344.245776,4,FALSE,0,0,8,85,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432,Amsterdam,Weekdays,Netherlands,Private
2,264.101422,2,FALSE,0,1,9,87,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103,Amsterdam,Weekdays,Netherlands,Private
3,433.529398,4,FALSE,0,1,9,90,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663,Amsterdam,Weekdays,Netherlands,Private
4,485.552926,2,TRUE,0,0,10,98,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508,Amsterdam,Weekdays,Netherlands,Private


### Convert 'TRUE' to 1 and 'FALSE' to 0 in the host_is_superhost column

In [10]:
# Convert 'TRUE' to 1 and 'FALSE' to 0 in the host_is_superhost column
combined_df['host_is_superhost'] = combined_df['host_is_superhost'].replace({'TRUE': 1, 'FALSE': 0})

# Display the updated DataFrame to verify
display(combined_df.head())

,price,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,city,day_type,country,room_type
0,194.033698,2,0,1,0,10,93,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772,Amsterdam,Weekdays,Netherlands,Private
1,344.245776,4,0,0,0,8,85,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432,Amsterdam,Weekdays,Netherlands,Private
2,264.101422,2,0,0,1,9,87,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103,Amsterdam,Weekdays,Netherlands,Private
3,433.529398,4,0,0,1,9,90,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663,Amsterdam,Weekdays,Netherlands,Private
4,485.552926,2,1,0,0,10,98,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508,Amsterdam,Weekdays,Netherlands,Private


### Instead of dummy variables, create a column where one can know if the host of the listing has only one, two, four, and more than four listings

In [11]:
combined_df[combined_df['biz'] == 0].head(15)

,price,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,city,day_type,country,room_type
0,194.033698,2,0,1,0,10,93,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772,Amsterdam,Weekdays,Netherlands,Private
1,344.245776,4,0,0,0,8,85,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432,Amsterdam,Weekdays,Netherlands,Private
4,485.552926,2,1,0,0,10,98,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508,Amsterdam,Weekdays,Netherlands,Private
5,552.808567,3,0,0,0,8,100,2,2.131420,1.904668,174.788957,9.255191,225.201662,15.692376,4.87699,52.38966,Amsterdam,Weekdays,Netherlands,Private
6,215.124317,2,0,0,0,10,94,1,1.881092,0.729747,200.167652,10.599010,242.765524,16.916251,4.91570,52.38296,Amsterdam,Weekdays,Netherlands,Private
7,2771.307384,4,1,0,0,10,100,3,1.686807,1.458404,208.808109,11.056528,272.313823,18.975219,4.88467,52.38749,Amsterdam,Weekdays,Netherlands,Entire Apt
8,1001.804420,4,0,0,0,9,96,2,3.719141,1.196112,106.226456,5.624761,133.876202,9.328686,4.86459,52.40175,Amsterdam,Weekdays,Netherlands,Entire Apt
9,276.521454,2,0,1,0,10,88,1,3.142361,0.924404,206.252861,10.921226,238.291258,16.604478,4.87600,52.34700,Amsterdam,Weekdays,Netherlands,Private
10,909.474375,2,0,0,0,10,96,1,1.009922,0.917115,409.858124,21.702260,555.114276,38.681161,4.87956,52.36953,Amsterdam,Weekdays,Netherlands,Entire Apt
11,319.640053,2,1,1,0,10,97,1,2.182707,1.590381,191.501339,10.140123,229.297401,15.977773,4.92496,52.37107,Amsterdam,Weekdays,Netherlands,Private


In [12]:
# Define conditions
conditions = [
    (combined_df['multi'] == 0) & (combined_df['biz'] == 0),  # Single listing
    (combined_df['multi'] == 1) & (combined_df['biz'] == 0),  # Multiple (2-4) listings but not business
    (combined_df['biz'] == 1)  # Business, more than four listings
]

# Define the corresponding choices for each condition
choices = ['Single', 'Few', 'Many']

# Create the new column using np.select, with the default set to 'Unknown' or leave as NaN
combined_df['listing_num'] = np.select(conditions, choices, default='Unknown')

# Drop the original room_shared and room_private columns if they are no longer needed
combined_df = combined_df.drop(columns=['multi', 'biz'])

# Display the updated DataFrame
display(combined_df.head())

,price,person_capacity,host_is_superhost,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,city,day_type,country,room_type,listing_num
0,194.033698,2,0,10,93,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772,Amsterdam,Weekdays,Netherlands,Private,Few
1,344.245776,4,0,8,85,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432,Amsterdam,Weekdays,Netherlands,Private,Single
2,264.101422,2,0,9,87,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103,Amsterdam,Weekdays,Netherlands,Private,Many
3,433.529398,4,0,9,90,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663,Amsterdam,Weekdays,Netherlands,Private,Many
4,485.552926,2,1,10,98,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508,Amsterdam,Weekdays,Netherlands,Private,Single


In [13]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51707 entries, 0 to 51706
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   price                       51707 non-null  float64
 1   person_capacity             51707 non-null  int64  
 2   host_is_superhost           51707 non-null  int64  
 3   cleanliness_rating          51707 non-null  int64  
 4   guest_satisfaction_overall  51707 non-null  int64  
 5   bedrooms                    51707 non-null  int64  
 6   dist                        51707 non-null  float64
 7   metro_dist                  51707 non-null  float64
 8   attr_index                  51707 non-null  float64
 9   attr_index_norm             51707 non-null  float64
 10  rest_index                  51707 non-null  float64
 11  rest_index_norm             51707 non-null  float64
 12  lng                         51707 non-null  float64
 13  lat                         517

## EDA